In [1]:
!pip install networkx
!pip install wikipedia-api
!pip install graphviz
!pip install pydot

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp37-none-any.whl size=13462 sha256=714efcc1bef18be735f05a77aba29ea0e474a6c58f74c58369aa68cb78c392bf
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api


In [248]:
import wikipediaapi
import networkx as nx
import matplotlib.pyplot as plt 
import pydot
import string
import numpy as np
import networkx.classes.function as nxcf

In [257]:
def valid1(x):
    if ':' not in x and 'List' not in x and 'disambiguation' not in x:
        for ch in x:
            if ch not in string.printable:
                return False
        return True
    return False


def valid2(x):
    if ':' not in x and 'List' not in x and 'disambiguation' not in x and x in L[:n]:
        for ch in x:
            if ch not in string.printable:
                return False
        return True
    return False

In [258]:
n = 40

In [259]:
page_name = 'Chemistry'
wiki_api = wikipediaapi.Wikipedia(language='en')
page = wiki_api.page(page_name)
D = dict.fromkeys(filter(valid1, page.links.keys()))
L = ['Chemistry'] + list(D.keys())
for name in L[:n]:
    page = wiki_api.page(name)
    D[name] = list(filter(valid2, page.links.keys()))

In [260]:
G = nx.DiGraph()
for name in L[:n]:
    if D[name] not in [[], None]:
        for inner_name in D[name]:
          if inner_name != name:
            G.add_edge(inner_name, name)
G.remove_nodes_from([node for node, degree in dict(G.degree()).items() if degree < 2])

In [265]:
# A[i][j] = 0 means that there is no edge between them
# A[i][j] = 1 means that the edge denotes j depends on i ; color = black
# A[i][j] = 2 means that the edge denotes that i is super-topic of j ; color = blue
deg = {node: degree for node, degree in G.degree()}

<class 'networkx.classes.reportviews.DiDegreeView'>


In [262]:
to_remove = []
edge_attrs = {}
set_list = [tuple(sorted(edge)) for edge in G.edges()]
duplicate_list = set(filter(lambda x: set_list.count(x) > 1, set_list))
for edge in duplicate_list:
  u, v = edge
  top = max(edge, key=lambda x: deg[x])
  bottom = min(edge, key=lambda x: deg[x])
  if top == bottom:
    top, bottom = u, v
  edge_attrs[(top, bottom)] = {'weight': 2, 'color': 'green'}
  to_remove.append((bottom, top))
nxcf.set_edge_attributes(G, edge_attrs)
G.remove_edges_from(to_remove)

In [263]:
cycles = list(nx.algorithms.cycles.simple_cycles(G))
print('nos cycles', len(cycles))
adj = nx.linalg.graphmatrix.adjacency_matrix(G).A
n_false_cycles = 0
chem_in_cycle = 0
for ind, cycle in enumerate(cycles):
  k = len(cycle)
  for i in range(k):
    if (cycle[i], cycle[(i+1)%k]) in nxcf.get_edge_attributes(G, 'weight'):
      n_false_cycles += 1
      break
  else:
    if 'Chemistry' in cycle:
      chem_in_cycle += 1
    else:
      print(cycle)
print('false cycles:', n_false_cycles)
print('True cycles that have Chemistry in them:', chem_in_cycle)

nos cycles 221
false cycles: 221
True cycles that have Chemistry in them: 0


In [264]:
graph = nx.drawing.nx_pydot.to_pydot(G)
graph.write_png('Chemistry_new_output_{}nodes_pruned.png'.format(n))